In [28]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torch.nn.functional  as F

import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

from torchinfo import summary

import numpy as np
import matplotlib.pyplot as plt

# Define relevant variables for the Machine Learning task
batch_size = 64
num_classes = 10
learning_rate = 0.01
num_epochs = 20

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("using device: %s" % device)

using device: cuda


In [29]:
def data_loader(data_dir,
                batch_size,
                random_seed=42,
                valid_size=0.1,
                shuffle=True,
                test=False):
  
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )

    # define transforms
    transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            normalize,
    ])

    if test:
        dataset = datasets.CIFAR10(
            root=data_dir, train=False,
            download=True, transform=transform,
        )

        data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=shuffle
        )

        return data_loader

    # load the dataset
    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler)
 
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler)

    return (train_loader, valid_loader)


# CIFAR10 dataset 
train_loader, valid_loader = data_loader(data_dir='./data', batch_size=batch_size)

test_loader = data_loader(data_dir='./data', batch_size=batch_size, test=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [30]:
# build a ResidualBlock
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels)
        )
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [31]:
# ResNet-34
# there are 4 blocks in the architecture,
# containing 3, 4, 6, and 3 layers respectively
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=num_classes):
        super().__init__()
        self.inplanes = 64  # in_channels
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # there are 4 blocks in the architecture,
        # containing 3, 4, 6, and 3 layers respectively
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
    
    # 构造由残差块ResidualBlock组成的模块
    # block: 残差块; planes: 输出通道数; blocks: 模块中残差块的数目
    def _make_layer(self, block, planes, blocks, stride):
        downsample = None
        # 第一个模块之后的每个模块在第一个残差块里将上一个模块的通道数翻倍（使用1x1卷积），并将高和宽减半
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes)
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample)) # 每个模块的第一个残差块
        self.inplanes = planes  # Note: 之后的残差块通道数相同
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x


In [32]:
model = ResNet(ResidualBlock, [3, 4, 6, 3])
# print(model)
summary(model=model, input_size=(1, 3, 224, 224), col_width=20, 
        col_names=['input_size', 'output_size', 'num_params', 'trainable'], 
        row_settings=['var_names'], verbose=0)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [1, 3, 224, 224]     [1, 10]              --                   True
├─Sequential (conv1)                     [1, 3, 224, 224]     [1, 64, 112, 112]    --                   True
│    └─Conv2d (0)                        [1, 3, 224, 224]     [1, 64, 112, 112]    9,472                True
│    └─BatchNorm2d (1)                   [1, 64, 112, 112]    [1, 64, 112, 112]    128                  True
│    └─ReLU (2)                          [1, 64, 112, 112]    [1, 64, 112, 112]    --                   --
├─MaxPool2d (maxpool)                    [1, 64, 112, 112]    [1, 64, 56, 56]      --                   --
├─Sequential (layer1)                    [1, 64, 56, 56]      [1, 64, 56, 56]      --                   True
│    └─ResidualBlock (0)                 [1, 64, 56, 56]      [1, 64, 56, 56]      --                   True
│    │    └─Sequen

In [33]:
model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)

# Train the model
total_step = len(train_loader)

for epoch in range(num_epochs):
    model.train()   # sets the module in training mode
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    model.eval()    # sets the module in evaluation mode
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
        print('Accuracy of the network on the {} validation images: {} %' 
              .format(5000, 100 * correct / total))

Epoch [1/20], Step [704/704], Loss: 1.5582
Accuracy of the network on the 5000 validation images: 53.94 %
Epoch [2/20], Step [704/704], Loss: 0.8758
Accuracy of the network on the 5000 validation images: 69.22 %
Epoch [3/20], Step [704/704], Loss: 0.4167
Accuracy of the network on the 5000 validation images: 75.98 %
Epoch [4/20], Step [704/704], Loss: 0.8871
Accuracy of the network on the 5000 validation images: 78.14 %
Epoch [5/20], Step [704/704], Loss: 1.6262
Accuracy of the network on the 5000 validation images: 80.02 %
Epoch [6/20], Step [704/704], Loss: 0.0743
Accuracy of the network on the 5000 validation images: 80.74 %
Epoch [7/20], Step [704/704], Loss: 0.1247
Accuracy of the network on the 5000 validation images: 77.8 %
Epoch [8/20], Step [704/704], Loss: 0.4560
Accuracy of the network on the 5000 validation images: 80.26 %
Epoch [9/20], Step [704/704], Loss: 0.1756
Accuracy of the network on the 5000 validation images: 81.8 %
Epoch [10/20], Step [704/704], Loss: 0.9592
Accu

In [34]:
model.eval()    # sets the module in evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %' 
          .format(10000, 100 * correct / total))

Accuracy of the network on the 10000 test images: 82.29 %


In [ ]:
# EXPORTING A MODEL FROM PYTORCH TO ONNX
model = model.to('cpu')
model.eval()
x = torch.randn(size=(1, 3, 224, 224), dtype=torch.float32, requires_grad=True)
torch.onnx.export(model,                                        # model being run
                  x,                                            # model input (or a tuple for multiple inputs)
                  "ResNet.onnx",                                # where to save the model (can be a file or file-like object)
                  export_params=True,                           # store the trained parameter weights inside the model file
                  opset_version=11,                             # the ONNX version to export the model to
                  do_constant_folding=True,                     # whether to execute constant folding for optimization
                  input_names = ['input'],                      # the model's input names
                  output_names = ['output'],                    # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},   # variable length axes
                                'output' : {0 : 'batch_size'}})